在模型训练部分，为了保证模型的真实效果，我们需要对模型进行一些调试和优化，主要分为以下五个环节：

- 计算分类准确率，观测模型训练效果。

    交叉熵损失函数只能作为优化目标，无法直接准确衡量模型的训练效果。准确率可以直接衡量训练效果，但由于其离散性质，不适合做为损失函数优化神经网络。
    
- 检查模型训练过程，识别潜在问题。

    如果模型的损失或者评估指标表现异常，我们通常需要打印模型每一层的输入和输出来定位问题，分析每一层的内容来获取错误的原因。
    
- 加入校验或测试，更好评价模型效果。

	理想的模型训练结果是在训练集和验证集上均有较高的准确率，如果训练集上的准确率高于验证集，说明网络训练程度不够；如果验证集的准确率高于训练集，可能是发生了	过拟合现象。通过在优化目标中加入正则化项的办法，可以解决过拟合的问题。
    
- 加入正则化项，避免模型过拟合。

- 可视化分析。

	用户不仅可以通过打印或使用matplotlib库作图，飞桨还集成了更专业的第三方绘图库tb-paddle，提供便捷的可视化分析。

## 1. 计算模型的分类准确率

飞桨提供了便利的计算分类准确率的API，使用fluid.layers.accuracy(input, label)可以直接计算准确率，该API的输入为预测的分类结果input和对应的标签label。

在下述代码中，我们在模型前向计算过程forward函数中计算分类准确率。

In [1]:
# 加载相关库
import os
import random
import paddle
import paddle.fluid as fluid
from paddle.fluid.dygraph.nn import Conv2D, Pool2D, FC
import numpy as np
from PIL import Image

import gzip
import json

# 定义数据集读取器
def load_data(mode='train'):

    # 读取数据文件
    datafile = './work/mnist.json.gz'
    print('loading mnist dataset from {} ......'.format(datafile))
    data = json.load(gzip.open(datafile))
    # 读取数据集中的训练集，验证集和测试集
    train_set, val_set, eval_set = data

    # 数据集相关参数，图片高度IMG_ROWS, 图片宽度IMG_COLS
    IMG_ROWS = 28
    IMG_COLS = 28
    # 根据输入mode参数决定使用训练集，验证集还是测试
    if mode == 'train':
        imgs = train_set[0]
        labels = train_set[1]
    elif mode == 'valid':
        imgs = val_set[0]
        labels = val_set[1]
    elif mode == 'eval':
        imgs = eval_set[0]
        labels = eval_set[1]
    # 获得所有图像的数量
    imgs_length = len(imgs)
    # 验证图像数量和标签数量是否一致
    assert len(imgs) == len(labels), \
          "length of train_imgs({}) should be the same as train_labels({})".format(
                  len(imgs), len(labels))

    index_list = list(range(imgs_length))

    # 读入数据时用到的batchsize
    BATCHSIZE = 100

    # 定义数据生成器
    def data_generator():
        # 训练模式下，打乱训练数据
        if mode == 'train':
            random.shuffle(index_list)
        imgs_list = []
        labels_list = []
        # 按照索引读取数据
        for i in index_list:
            # 读取图像和标签，转换其尺寸和类型
            img = np.reshape(imgs[i], [1, IMG_ROWS, IMG_COLS]).astype('float32')
            label = np.reshape(labels[i], [1]).astype('int64')
            imgs_list.append(img) 
            labels_list.append(label)
            # 如果当前数据缓存达到了batch size，就返回一个批次数据
            if len(imgs_list) == BATCHSIZE:
                yield np.array(imgs_list), np.array(labels_list)
                # 清空数据缓存列表
                imgs_list = []
                labels_list = []

        # 如果剩余数据的数目小于BATCHSIZE，
        # 则剩余数据一起构成一个大小为len(imgs_list)的mini-batch
        if len(imgs_list) > 0:
            yield np.array(imgs_list), np.array(labels_list)

    return data_generator


# 定义模型结构
class MNIST(fluid.dygraph.Layer):
     def __init__(self, name_scope):
         super(MNIST, self).__init__(name_scope)
         name_scope = self.full_name()
         # 定义卷积层，输出通道20，卷积核大小为5，步长为1，padding为2，使用relu激活函数
         self.conv1 = Conv2D(name_scope, num_filters=20, filter_size=5, stride=1, padding=2, act='relu')
         # 定义池化层，池化核为2，采用最大池化方式
         self.pool1 = Pool2D(name_scope, pool_size=2, pool_stride=2, pool_type='max')
         # 定义卷积层，输出通道20，卷积核大小为5，步长为1，padding为2，使用relu激活函数
         self.conv2 = Conv2D(name_scope, num_filters=20, filter_size=5, stride=1, padding=2, act='relu')
         # 定义池化层，池化核为2，采用最大池化方式
         self.pool2 = Pool2D(name_scope, pool_size=2, pool_stride=2, pool_type='max')
         # 定义全连接层，输出节点数为10，激活函数使用softmax
         self.fc = FC(name_scope, size=10, act='softmax')
         
    # 定义网络的前向计算过程
     def forward(self, inputs, label=None):
         x = self.conv1(inputs)
         x = self.pool1(x)
         x = self.conv2(x)
         x = self.pool2(x)
         x = self.fc(x)
         if label is not None:
             acc = fluid.layers.accuracy(input=x, label=label)
             return x, acc
         else:
             return x

#调用加载数据的函数
train_loader = load_data('train')
    
#在使用GPU机器时，可以将use_gpu变量设置成True
use_gpu = False
place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()

with fluid.dygraph.guard(place):
    model = MNIST("mnist")
    model.train() 
    
    #四种优化算法的设置方案，可以逐一尝试效果
    optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.MomentumOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.AdagradOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.AdamOptimizer(learning_rate=0.01)
    
    EPOCH_NUM = 5
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据
            image_data, label_data = data
            image = fluid.dygraph.to_variable(image_data)
            label = fluid.dygraph.to_variable(label_data)
            
            #前向计算的过程，同时拿到模型输出值和分类准确率
            predict, avg_acc = model(image, label)
            
            #计算损失，取一个批次样本损失的平均值
            loss = fluid.layers.cross_entropy(predict, label)
            avg_loss = fluid.layers.mean(loss)
            
            #每训练了200批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(),avg_acc.numpy()))
            
            #后向传播，更新参数的过程
            avg_loss.backward()
            optimizer.minimize(avg_loss)
            model.clear_gradients()

    #保存模型参数
    fluid.save_dygraph(model.state_dict(), 'mnist')

loading mnist dataset from ./work/mnist.json.gz ......
epoch: 0, batch: 0, loss is: [2.6924028], acc is [0.13]
epoch: 0, batch: 200, loss is: [0.43108827], acc is [0.9]
epoch: 0, batch: 400, loss is: [0.28437912], acc is [0.91]
epoch: 1, batch: 0, loss is: [0.18557501], acc is [0.94]
epoch: 1, batch: 200, loss is: [0.2572117], acc is [0.92]
epoch: 1, batch: 400, loss is: [0.23375131], acc is [0.96]
epoch: 2, batch: 0, loss is: [0.2376423], acc is [0.92]
epoch: 2, batch: 200, loss is: [0.2346335], acc is [0.94]
epoch: 2, batch: 400, loss is: [0.12313923], acc is [0.96]
epoch: 3, batch: 0, loss is: [0.11195458], acc is [0.97]
epoch: 3, batch: 200, loss is: [0.06257623], acc is [0.99]
epoch: 3, batch: 400, loss is: [0.09305647], acc is [0.98]
epoch: 4, batch: 0, loss is: [0.11276975], acc is [0.96]
epoch: 4, batch: 200, loss is: [0.11841835], acc is [0.96]
epoch: 4, batch: 400, loss is: [0.17532296], acc is [0.93]


## 2. 检查模型训练过程，识别潜在训练问题

不同于某些深度学习框架的高层API，使用飞桨动态图编程可以方便的查看和调试训练的执行过程。在网络定义的Forward函数中，可以打印每一层输入输出的尺寸，以及每层网络的参数。通过查看这些信息，不仅可以让我们可以更好的理解训练的执行过程，还可以发现潜在问题，或者启发继续优化的思路。



In [3]:
# 定义模型结构
class MNIST(fluid.dygraph.Layer):
     def __init__(self, name_scope):
         super(MNIST, self).__init__(name_scope)
         name_scope = self.full_name()
         self.conv1 = Conv2D(name_scope, num_filters=20, filter_size=5, stride=1, padding=2)
         self.pool1 = Pool2D(name_scope, pool_size=2, pool_stride=2, pool_type='max')
         self.conv2 = Conv2D(name_scope, num_filters=20, filter_size=5, stride=1, padding=2)
         self.pool2 = Pool2D(name_scope, pool_size=2, pool_stride=2, pool_type='max')
         self.fc = FC(name_scope, size=10, act='softmax')
     
     #加入对每一层输入和输出的尺寸和数据内容的打印，根据check参数决策是否打印每层的参数和输出尺寸
     def forward(self, inputs, label=None, check_shape=False, check_content=False):
         # 给不同层的输出不同命名，方便调试
         outputs1 = self.conv1(inputs)
         outputs2 = self.pool1(outputs1)
         outputs3 = self.conv2(outputs2)
         outputs4 = self.pool2(outputs3)
         outputs5 = self.fc(outputs4)
         
         # 选择是否打印神经网络每层的参数尺寸和输出尺寸，验证网络结构是否设置正确
         if check_shape:
             # 打印每层网络设置的超参数-卷积核尺寸，卷积步长，卷积padding，池化核尺寸
             print("\n########## print network layer's superparams ##############")
             print("conv1-- kernel_size:{}, padding:{}, stride:{}".format(self.conv1.weight.shape, self.conv1._padding, self.conv1._stride))
             print("conv2-- kernel_size:{}, padding:{}, stride:{}".format(self.conv2.weight.shape, self.conv2._padding, self.conv2._stride))
             print("pool1-- pool_type:{}, pool_size:{}, pool_stride:{}".format(self.pool1._pool_type, self.pool1._pool_size, self.pool1._pool_stride))
             print("pool2-- pool_type:{}, poo2_size:{}, pool_stride:{}".format(self.pool2._pool_type, self.pool2._pool_size, self.pool2._pool_stride))
             print("fc-- weight_size:{}, bias_size_{}, activation:{}".format(self.fc.weight.shape, self.fc.bias.shape, self.fc._act))
             
             # 打印每层的输出尺寸
             print("\n########## print shape of features of every layer ###############")
             print("inputs_shape: {}".format(inputs.shape))
             print("outputs1_shape: {}".format(outputs1.shape))
             print("outputs2_shape: {}".format(outputs2.shape))
             print("outputs3_shape: {}".format(outputs3.shape))
             print("outputs4_shape: {}".format(outputs4.shape))
             print("outputs5_shape: {}".format(outputs5.shape))
             
         # 选择是否打印训练过程中的参数和输出内容，可用于训练过程中的调试
         if check_content:
            # 打印卷积层的参数-卷积核权重，权重参数较多，此处只打印部分参数
             print("\n########## print convolution layer's kernel ###############")
             print("conv1 params -- kernel weights:", self.conv1.weight[0][0])
             print("conv2 params -- kernel weights:", self.conv2.weight[0][0])

             # 创建随机数，随机打印某一个通道的输出值
             idx1 = np.random.randint(0, outputs1.shape[1])
             idx2 = np.random.randint(0, outputs3.shape[1])
             # 打印卷积-池化后的结果，仅打印batch中第一个图像对应的特征
             print("\nThe {}th channel of conv1 layer: ".format(idx1), outputs1[0][idx1])
             print("The {}th channel of conv2 layer: ".format(idx2), outputs3[0][idx2])
             print("The output of last layer:", outputs5[0], '\n')
            
        # 如果label不是None，则计算分类精度并返回
         if label is not None:
             acc = fluid.layers.accuracy(input=outputs5, label=label)
             return outputs5, acc
         else:
             return outputs5

    
#在使用GPU机器时，可以将use_gpu变量设置成True
use_gpu = False
place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()

with fluid.dygraph.guard(place):
    model = MNIST("mnist")
    model.train() 
    
    #四种优化算法的设置方案，可以逐一尝试效果
    optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.MomentumOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.AdagradOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.AdamOptimizer(learning_rate=0.01)
    
    EPOCH_NUM = 1
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据，变得更加简洁
            image_data, label_data = data
            image = fluid.dygraph.to_variable(image_data)
            label = fluid.dygraph.to_variable(label_data)
            
            #前向计算的过程，同时拿到模型输出值和分类准确率
            if batch_id == 0 and epoch_id==0:
                # 打印模型参数和每层输出的尺寸
                predict, acc = model(image, label, check_shape=True, check_content=False)
            elif batch_id==401:
                # 打印模型参数和每层输出的值
                predict, avg_acc = model(image, label, check_shape=False, check_content=True)
            else:
                predict, avg_acc = model(image, label)
            
            #计算损失，取一个批次样本损失的平均值
            loss = fluid.layers.cross_entropy(predict, label)
            avg_loss = fluid.layers.mean(loss)
            
            #每训练了100批次的数据，打印下当前Loss的情况
            if batch_id % 200 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(),avg_acc.numpy()))
            
            #后向传播，更新参数的过程
            avg_loss.backward()
            optimizer.minimize(avg_loss)
            model.clear_gradients()

    #保存模型参数
    fluid.save_dygraph(model.state_dict(), 'mnist')


########## print network layer's superparams ##############
conv1-- kernel_size:[20, 1, 5, 5], padding:[2, 2], stride:[1, 1]
conv2-- kernel_size:[20, 20, 5, 5], padding:[2, 2], stride:[1, 1]
pool1-- pool_type:max, pool_size:[2, 2], pool_stride:[2, 2]
pool2-- pool_type:max, poo2_size:[2, 2], pool_stride:[2, 2]
fc-- weight_size:[980, 10], bias_size_[10], activation:softmax

########## print shape of features of every layer ###############
inputs_shape: [100, 1, 28, 28]
outputs1_shape: [100, 20, 28, 28]
outputs2_shape: [100, 20, 14, 14]
outputs3_shape: [100, 20, 14, 14]
outputs4_shape: [100, 20, 7, 7]
outputs5_shape: [100, 10]
epoch: 0, batch: 0, loss is: [2.3864117], acc is [0.06]
epoch: 0, batch: 200, loss is: [0.28423238], acc is [0.89]
epoch: 0, batch: 400, loss is: [0.25175756], acc is [0.94]

########## print convolution layer's kernel ###############
conv1 params -- kernel weights: name tmp_7629, dtype: VarType.FP32 shape: [5, 5] 	lod: {}
	dim: 5, 5
	layout: NCHW
	dtype: float
	da

## 3. 加入校验或测试，更好评价模型效果 

在训练过程中，我们会发现模型在训练样本集上的损失在不断减小。但这是否代表模型在未来的应用场景上依然有效？为了验证模型的有效性，通常将样本集合分成三份，训练集、校验集和测试集。

- **训练集**。用于训练模型的参数，即训练过程中主要完成的工作。
- **校验集**。用于对模型超参数的选择，比如网络结构的调整、正则化项权重的选择等。
- **测试集**。用于模拟模型在应用后的真实效果。因为测试集没有参与任何模型优化或参数训练的工作，所以它对模型来说是完全未知的样本。在不以校验数据优化网络结构或模型超参数时，校验数据和测试数据的效果是类似的，均更真实的反映模型效果。

如下程序读取上一步训练保存的模型参数，读取校验数据集，测试模型在校验数据集上的效果。从测试的效果来看，模型在从来没有见过的数据集上依然有？%的准确率，证明它是有预测效果的。

In [ ]:
with fluid.dygraph.guard():
    print('start evaluation .......')
    #加载模型参数
    model = MNIST("mnist")
    model_state_dict, _ = fluid.load_dygraph('mnist')
    model.load_dict(model_state_dict)

    model.eval()
    eval_loader = load_data('eval')

    acc_set = []
    avg_loss_set = []
    for batch_id, data in enumerate(eval_loader()):
        x_data, y_data = data
        img = fluid.dygraph.to_variable(x_data)
        label = fluid.dygraph.to_variable(y_data)
        prediction, acc = model(img, label)
        loss = fluid.layers.cross_entropy(input=prediction, label=label)
        avg_loss = fluid.layers.mean(loss)
        acc_set.append(float(acc.numpy()))
        avg_loss_set.append(float(avg_loss.numpy()))
    
    #计算多个batch的平均损失和准确率
    acc_val_mean = np.array(acc_set).mean()
    avg_loss_val_mean = np.array(avg_loss_set).mean()

    print('loss={}, acc={}'.format(avg_loss_val_mean, acc_val_mean))

start evaluation .......
loading mnist dataset from ./work/mnist.json.gz ......
loss=0.09713039316236972, acc=0.9705000025033951


## 4. 加入正则化项，避免模型过拟合
   
对于样本量有限、但需要使用强大模型的复杂任务，模型很容易出现过拟合的表现，即在训练集上的损失小，在校验集或测试集上的损失较大。关于过拟合的详细理论，参考《机器学习的思考故事》课程。

为了避免模型过拟合，在没有扩充样本量的可能下，只能降低模型的复杂度。降低模型的复杂度，可以通过限制参数的数量或可能取值（参数值尽量小）实现。具体来说，在模型的优化目标（损失）中人为加入对参数规模的惩罚项。当参数越多或取值越大时，该惩罚项就越大。通过调整惩罚项的权重系数，可以使模型在“尽量减少训练损失”和“保持模型的泛化能力”之间取得平衡。泛化能力表示模型在没有见过的样本上依然有效。正则化项的存在，增加了模型在训练集上的损失。

飞桨框架支持为所有参数加上统一的正则化项，也支持为特定的参数添加正则化项。前者的实现如下代码所示，仅在优化器中设置regularization参数即可实现。使用参数regularization_coeff调节正则化项的权重，权重越大时，对模型复杂度的惩罚越高。

In [ ]:
with fluid.dygraph.guard():
    model = MNIST("mnist")
    model.train() 
    
    #四种优化算法的设置方案，可以逐一尝试效果
    #optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.MomentumOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.AdagradOptimizer(learning_rate=0.01)
    #optimizer = fluid.optimizer.AdamOptimizer(learning_rate=0.01)
    
    #各种优化算法均可以加入正则化项，避免过拟合，参数regularization_coeff调节正则化项的权重
    #optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01, regularization=fluid.regularizer.L2Decay(regularization_coeff=0.1))
    optimizer = fluid.optimizer.AdamOptimizer(learning_rate=0.01, regularization=fluid.regularizer.L2Decay(regularization_coeff=0.1))
    
    EPOCH_NUM = 10
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据，变得更加简洁
            image_data, label_data = data
            image = fluid.dygraph.to_variable(image_data)
            label = fluid.dygraph.to_variable(label_data)
            
            #前向计算的过程，同时拿到模型输出值和分类准确率
            predict, avg_acc = model(image, label)

            #计算损失，取一个批次样本损失的平均值
            loss = fluid.layers.cross_entropy(predict, label)
            avg_loss = fluid.layers.mean(loss)
            
            #每训练了100批次的数据，打印下当前Loss的情况
            if batch_id % 100 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(),avg_acc.numpy()))
            
            #后向传播，更新参数的过程
            avg_loss.backward()
            optimizer.minimize(avg_loss)
            model.clear_gradients()

    #保存模型参数
    fluid.save_dygraph(model.state_dict(), 'mnist')

epoch: 0, batch: 0, loss is: [2.5719237], acc is [0.2]
epoch: 0, batch: 100, loss is: [0.2886358], acc is [0.93]
epoch: 0, batch: 200, loss is: [0.3907683], acc is [0.91]
epoch: 0, batch: 300, loss is: [0.34717605], acc is [0.91]
epoch: 0, batch: 400, loss is: [0.29703715], acc is [0.92]
epoch: 1, batch: 0, loss is: [0.35397568], acc is [0.88]
epoch: 1, batch: 100, loss is: [0.3475823], acc is [0.89]
epoch: 1, batch: 200, loss is: [0.31529555], acc is [0.94]
epoch: 1, batch: 300, loss is: [0.4401128], acc is [0.86]
epoch: 1, batch: 400, loss is: [0.43345112], acc is [0.88]
epoch: 2, batch: 0, loss is: [0.30035377], acc is [0.93]
epoch: 2, batch: 100, loss is: [0.19205013], acc is [0.96]
epoch: 2, batch: 200, loss is: [0.35591784], acc is [0.87]
epoch: 2, batch: 300, loss is: [0.2924868], acc is [0.9]
epoch: 2, batch: 400, loss is: [0.26790595], acc is [0.95]
epoch: 3, batch: 0, loss is: [0.23955241], acc is [0.94]
epoch: 3, batch: 100, loss is: [0.24024098], acc is [0.92]
epoch: 3, bat

## 5. 可视化分析

训练模型时，我们经常需要观察模型的评价指标，分析模型的优化过程，以确保训练是有效的。如之前的案例所示，使用轻量级的PLT库作图各种指标是非常简单的。

### 使用Matplotlib库画出损失随训练下降的曲线图

首先将训练的批次编号作为X轴坐标，该批次的训练损失作为Y轴坐标。使用两个列表变量存储对应的批次编号(iters=[])和训练损失(losses=[])，并将两份数据以参数形式导入PLT的横纵坐标（    plt.xlabel("iter", fontsize=14)，plt.ylabel("loss", fontsize=14)）。最后，调用plt.plot()函数即可完成作图。

In [ ]:
#引入matplotlib库
import matplotlib.pyplot as plt

with fluid.dygraph.guard(place):
    model = MNIST("mnist")
    model.train() 
    
    #四种优化算法的设置方案，可以逐一尝试效果
    optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
    
    EPOCH_NUM = 10
    iter=0
    iters=[]
    losses=[]
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据，变得更加简洁
            image_data, label_data = data
            image = fluid.dygraph.to_variable(image_data)
            label = fluid.dygraph.to_variable(label_data)
            
            #前向计算的过程，同时拿到模型输出值和分类准确率
            predict, avg_acc = model(image, label)

            #计算损失，取一个批次样本损失的平均值
            loss = fluid.layers.cross_entropy(predict, label)
            avg_loss = fluid.layers.mean(loss)
            
            #每训练了100批次的数据，打印下当前Loss的情况
            if batch_id % 100 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(),avg_acc.numpy()))
                iters.append(iter)
                losses.append(avg_loss.numpy())
                iter = iter + 100

            #后向传播，更新参数的过程
            avg_loss.backward()
            optimizer.minimize(avg_loss)
            model.clear_gradients()

    #保存模型参数
    fluid.save_dygraph(model.state_dict(), 'mnist')

    #画出训练过程中Loss的变化曲线
    plt.figure()
    plt.title("train loss", fontsize=24)
    plt.xlabel("iter", fontsize=14)
    plt.ylabel("loss", fontsize=14)
    plt.plot(iters, losses,color='red',label='train loss') 
    plt.grid()
    plt.show()

epoch: 0, batch: 0, loss is: [2.8867545], acc is [0.04]
epoch: 0, batch: 100, loss is: [0.81941694], acc is [0.81]
epoch: 0, batch: 200, loss is: [0.4344087], acc is [0.89]
epoch: 0, batch: 300, loss is: [0.24528305], acc is [0.93]
epoch: 0, batch: 400, loss is: [0.22761887], acc is [0.94]
epoch: 1, batch: 0, loss is: [0.21595153], acc is [0.93]
epoch: 1, batch: 100, loss is: [0.21793167], acc is [0.96]
epoch: 1, batch: 200, loss is: [0.30391282], acc is [0.9]
epoch: 1, batch: 300, loss is: [0.18076742], acc is [0.96]
epoch: 1, batch: 400, loss is: [0.12867425], acc is [0.97]
epoch: 2, batch: 0, loss is: [0.12843314], acc is [0.97]
epoch: 2, batch: 100, loss is: [0.10542139], acc is [0.97]
epoch: 2, batch: 200, loss is: [0.14310229], acc is [0.96]
epoch: 2, batch: 300, loss is: [0.17816946], acc is [0.94]
epoch: 2, batch: 400, loss is: [0.05590251], acc is [0.98]
epoch: 3, batch: 0, loss is: [0.15483671], acc is [0.97]
epoch: 3, batch: 100, loss is: [0.1392599], acc is [0.96]
epoch: 3,

<Figure size 640x480 with 1 Axes>

### 使用tb-paddle可视化分析

如果期望使用更加专业的作图工具，可以尝试tb-paddle。tb-paddle能够有效地展示飞桨框架在运行过程中的计算图、各种指标随着时间的变化趋势以及训练中使用到的数据信息。tb-paddle的使用也并不复杂，可分为如下四个主要步骤。

１.　步骤1：引入tb_paddle库，定义作图数据存储位置（供第3步使用），本案例的路径是“log/data”。
```
from tb_paddle import SummaryWriter
data_writer = SummaryWriter(logdir="log/data")
```
２.　步骤2：在训练过程中插入作图语句。当每100个batch训练完成后，将当前损失作为一个新增的数据点(scalar_x和loss的映射对)存储到第一步设置的文件中。使用变量scalar_x记录下已经训练的批次数，作为作图的X轴坐标。

```
data_writer.add_scalar("train/loss", avg_loss.numpy(), scalar_x)
data_writer.add_scalar("train/accuracy", avg_acc.numpy(), scalar_x)
scalar_x = scalar_x + 100
```

In [ ]:
#引入Tensorboard库，并设定保存作图数据的文件位置
from tb_paddle import SummaryWriter
data_writer = SummaryWriter(logdir="log/data")

with fluid.dygraph.guard(place):
    model = MNIST("mnist")
    model.train() 
    
    #四种优化算法的设置方案，可以逐一尝试效果
    optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.01)
    
    EPOCH_NUM = 10
    iter = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            #准备数据，变得更加简洁
            image_data, label_data = data
            image = fluid.dygraph.to_variable(image_data)
            label = fluid.dygraph.to_variable(label_data)
            
            #前向计算的过程，同时拿到模型输出值和分类准确率
            predict, avg_acc = model(image, label)

            #计算损失，取一个批次样本损失的平均值
            loss = fluid.layers.cross_entropy(predict, label)
            avg_loss = fluid.layers.mean(loss)
            
            #每训练了100批次的数据，打印下当前Loss的情况
            if batch_id % 100 == 0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), avg_acc.numpy()))
                data_writer.add_scalar("train/loss", avg_loss.numpy(), iter)
                data_writer.add_scalar("train/accuracy", avg_acc.numpy(), iter)
                iter = iter + 100

            #后向传播，更新参数的过程
            avg_loss.backward()
            optimizer.minimize(avg_loss)
            model.clear_gradients()

    #保存模型参数
    fluid.save_dygraph(model.state_dict(), 'mnist')

３.　步骤3：命令行启动 tensorboard

使用“tensorboard --logdir [数据文件所在文件夹路径] 的命令启动Tensor board。在Tensor board启动后，命令行会打印出可用浏览器查阅图形结果的网址。
``` 
$ tensorboard --logdir log/data
```

４.　步骤4：打开浏览器，查看作图结果
查阅的网址在第三步的启动命令后会打印出来（如TensorBoard 2.0.0 at http://localhost:6006/），将该网址输入浏览器地址栏刷新页面的效果如下图所示。除了右侧对数据点的作图外，左侧还有一个控制板，可以调整诸多作图的细节。


<img src="https://ai-studio-static-online.cdn.bcebos.com/0f1926d02e4d49ff88ff088e65d11bc851d37db0a4d048b1b2725a206fb55152" width="600" hegiht="400" align=center />

图：tb-paddle的作图示例